In [ ]:
import json
import datetime
import numpy as np
from flask import Flask, request, jsonify, render_template
import torch
from torch import nn
import joblib
import os
import pandas as pd

app = Flask(__name__)
log_path = 'data/log.json'

# Initialize empty data log
if os.path.exists(log_path):
    try:
        with open(log_path, 'r') as f:
            data_log = json.load(f)
    except:
        data_log = []
else:
    data_log = []
    # Create directories if they don't exist
    os.makedirs('data', exist_ok=True)
    # Initialize log file with empty array
    with open(log_path, 'w') as f:
        json.dump(data_log, f)

# Load model class and initialize
class StressNN(nn.Module):
    def __init__(self):
        super(StressNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 64), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(64, 32), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(32, 16), nn.ReLU(), nn.Linear(16, 2)
        )
    
    def forward(self, x):
        return self.net(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = StressNN().to(device)

# Load model and scaler
try:
    model.load_state_dict(torch.load('model/stress_model.pt')['model_state_dict'])
    model.eval()
    scaler = joblib.load('model/scaler.pkl')
    print("Model and scaler loaded successfully!")
except Exception as e:
    print(f"Loading error: {str(e)}")
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.mean_ = np.zeros(3)
    scaler.scale_ = np.ones(3)
    print("Using default scaler since model loading failed.")

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/update', methods=['POST'])
def update_data():
    global data_log
    try:
        content = request.json
        print("Received:", content)
        
        # Store raw values directly
        entry = {
            'time': datetime.datetime.now().isoformat(),
            'eda': float(content['eda_raw']),  # Store raw values
            'temp': float(content['temp_raw']),
            'hr': float(content['hr_raw']),
            'stressed': 0  # Will be updated after prediction
        }
        
        # Prepare features for prediction (use mapped values)
        features = pd.DataFrame([[
            float(content['eda']),
            float(content['temp']),
            float(content['hr'])
        ]], columns=['EDA', 'TEMP', 'HR'])
        
        # Predict stress
        try:
            with torch.no_grad():
                features_scaled = scaler.transform(features)
                features_tensor = torch.tensor(features_scaled, dtype=torch.float32).to(device)
                output = model(features_tensor)
                entry['stressed'] = int(torch.argmax(output, dim=1).item())
                print(f"Prediction successful: stressed={entry['stressed']}")
        except Exception as model_error:
            print(f"Model prediction error: {str(model_error)}")
            # Use a default value in case of prediction error
            entry['stressed'] = 0
        
        data_log.append(entry)
        with open(log_path, 'w') as f:
            json.dump(data_log, f)
        
        print(f"Saved entry. Total entries: {len(data_log)}")
        return jsonify({'status': 'success', 'stressed': entry['stressed']})
        
    except Exception as e:
        print(f"Error in update_data: {str(e)}")
        return jsonify({'status': 'error', 'message': str(e)}), 400

@app.route('/data')
def get_data():
    # Load data from file to ensure we have latest
    try:
        with open(log_path, 'r') as f:
            latest_data = json.load(f)
        return jsonify(latest_data)
    except Exception as e:
        print(f"Error reading log file: {str(e)}")
        return jsonify([])

def run_server():
    os.makedirs('model', exist_ok=True)
    os.makedirs('data', exist_ok=True)
    
    # Ensure log file exists
    if not os.path.exists(log_path):
        with open(log_path, 'w') as f:
            json.dump([], f)
    
    app.run(host='0.0.0.0', port=5000, debug=False)

if __name__ == '__main__':
    run_server()

Model and scaler loaded successfully!
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.20.10.10:5000
Press CTRL+C to quit
172.20.10.10 - - [12/May/2025 09:56:58] "GET /data HTTP/1.1" 200 -


Error reading log file: Expecting value: line 1 column 1 (char 0)


172.20.10.10 - - [12/May/2025 09:57:02] "GET / HTTP/1.1" 200 -
172.20.10.10 - - [12/May/2025 09:57:02] "GET /static/style.css HTTP/1.1" 304 -
172.20.10.10 - - [12/May/2025 09:57:02] "GET /static/script.js HTTP/1.1" 304 -
172.20.10.10 - - [12/May/2025 09:57:02] "GET /data HTTP/1.1" 200 -


Error reading log file: Expecting value: line 1 column 1 (char 0)


...

172.20.10.10 - - [12/May/2025 09:58:31] "GET /data HTTP/1.1" 200 -
172.20.10.10 - - [12/May/2025 09:58:31] "GET /data HTTP/1.1" 200 -


Error reading log file: Expecting value: line 1 column 1 (char 0)
Error reading log file: Expecting value: line 1 column 1 (char 0)


172.20.10.10 - - [12/May/2025 09:59:31] "GET /data HTTP/1.1" 200 -
172.20.10.10 - - [12/May/2025 09:59:31] "GET /data HTTP/1.1" 200 -


Error reading log file: Expecting value: line 1 column 1 (char 0)
Error reading log file: Expecting value: line 1 column 1 (char 0)


172.20.10.10 - - [12/May/2025 10:00:31] "GET /data HTTP/1.1" 200 -
172.20.10.10 - - [12/May/2025 10:00:31] "GET /data HTTP/1.1" 200 -
172.20.10.10 - - [12/May/2025 10:01:31] "GET /data HTTP/1.1" 200 -
172.20.10.10 - - [12/May/2025 10:01:31] "GET /data HTTP/1.1" 200 -
172.20.10.10 - - [12/May/2025 10:02:31] "GET /data HTTP/1.1" 200 -
172.20.10.10 - - [12/May/2025 10:02:31] "GET /data HTTP/1.1" 200 -
